Implementing MCTS algorithm for a simple game in which the player has to guess the number that the computer has randomly chosen between 1 and 100.

In [11]:
import random
import math

In [12]:
class GameState:
    def __init__(self):
        self.target_number = random.randint(1, 100)
        self.attempts = 0

    def is_terminal(self):
        return self.attempts >= 10 or self.attempts < 0

    def get_legal_actions(self):
        return list(range(1, 101))

    def perform_action(self, action):
        self.attempts += 1
        if action == self.target_number:
            return 1  # Player wins
        elif action < self.target_number:
            return 0  # Guess is too low
        else:
            return -1  # Guess is too high

In [13]:
class Node:
    def __init__(self, state, parent=None):
        self.state = state
        self.children = []
        self.visits = 0
        self.total_reward = 0
        self.parent = parent

In [14]:
def UCB1(node, parent_visits):
    if node.visits == 0:
        return float('inf')
    return (node.total_reward / node.visits) + 1.41 * math.sqrt(math.log(parent_visits) / node.visits)

In [15]:
def selection(node):
    return max(node.children, key=lambda child: UCB1(child, node.visits))

In [16]:
def expansion(node):
    legal_actions = node.state.get_legal_actions()
    untried_actions = [action for action in legal_actions if not any(child.state.attempts == action for child in node.children)]
    if untried_actions:
        action = random.choice(untried_actions)
        new_state = GameState()
        new_state.attempts = action
        child_node = Node(new_state, parent=node) 
        node.children.append(child_node)
        return child_node
    else:
        return random.choice(node.children)

In [17]:
def simulation(node):
    state = GameState()
    state.attempts = node.state.attempts
    while not state.is_terminal():
        legal_actions = state.get_legal_actions()
        action = random.choice(legal_actions)
        result = state.perform_action(action)
        if result == 1:
            return 1  # Player wins
    return 0  # Player loses

In [18]:
def backpropagation(node, reward):
    while node is not None:
        node.visits += 1
        node.total_reward += reward
        node = node.parent

In [19]:
def mcts(root_state, max_iterations):
    root_node = Node(root_state)
    for _ in range(max_iterations):
        node = root_node

        # Selection phase
        while not node.state.is_terminal() and node.children:
            node = selection(node)

        # Expansion phase
        if not node.state.is_terminal():
            node = expansion(node)

        # Simulation phase
        reward = simulation(node)

        # Backpropagation phase
        backpropagation(node, reward)

    # Select the best action to take from the root node
    best_action = max(root_node.children, key=lambda child: child.visits).state.attempts
    return best_action

In [23]:
if __name__ == "__main__":
    game_state = GameState()
    max_iterations = 1000

    while not game_state.is_terminal():
        best_action = mcts(game_state, max_iterations)
        result = game_state.perform_action(best_action)

        if result == 1:
            print(f"Congratulations! You guessed the correct number {game_state.target_number} in {game_state.attempts} attempts!")
            break
        elif result == 0:
            print(f"Try a higher number.")
        else:
            print(f"Try a lower number.")
    
    if game_state.is_terminal() and result != 1:
        print(f"Sorry, you didn't guess the correct number. It was {game_state.target_number}.")

Try a higher number.
Try a higher number.
Try a higher number.
Try a higher number.
Try a higher number.
Try a lower number.
Try a higher number.
Try a lower number.
Try a lower number.
Try a higher number.
Sorry, you didn't guess the correct number. It was 53.
